In [3]:
from pyspark.sql import SparkSession, functions as func, Row
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType, LongType

In [4]:
# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/06/17 21:22:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
movie_schema = StructType([
    StructField("uid", IntegerType(), True),
    StructField("mid", IntegerType(), True),
    StructField("rating", IntegerType(), True),
    StructField("timestamp", LongType(), True)
])

In [6]:
movieDf = spark.read.option("sep", "\t").schema(movie_schema).csv("ml-100k/u.data")

In [32]:
movie = movieDf.groupBy("mid").count()

In [34]:
movie = movie.orderBy(func.desc("count"))

In [35]:
movie.show(5)

+---+-----+
|mid|count|
+---+-----+
| 50|  583|
|258|  509|
|100|  508|
|181|  507|
|294|  485|
+---+-----+
only showing top 5 rows



---------

In [10]:
import codecs

In [11]:
def load_movies():
    movie_names = {}
    with codecs.open("ml-100k/u.item", 'r', encoding='ISO-8859-1') as file:
        for line in file:
            fields = line.split('|')
            movie_names[int(fields[0])] = fields[1]
    return movie_names

In [12]:
movie_name = spark.sparkContext.broadcast(load_movies())

In [13]:
def lookup_name(mid):
    return movie_name.value[mid]

In [16]:
lookupUDF = func.udf(lookup_name)

In [18]:
movie_with_name = movie.withColumn("movie_title", lookupUDF(func.col("mid")) )

In [19]:
movie_with_name.show(10, False)

+---+-----+-----------------------------+
|mid|count|movie_title                  |
+---+-----+-----------------------------+
|50 |583  |Star Wars (1977)             |
|258|509  |Contact (1997)               |
|100|508  |Fargo (1996)                 |
|181|507  |Return of the Jedi (1983)    |
|294|485  |Liar Liar (1997)             |
|286|481  |English Patient, The (1996)  |
|288|478  |Scream (1996)                |
|1  |452  |Toy Story (1995)             |
|300|431  |Air Force One (1997)         |
|121|429  |Independence Day (ID4) (1996)|
+---+-----+-----------------------------+
only showing top 10 rows



In [1]:
!{"head -n 5 ml-100k/u.item"}

1|Toy Story (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Toy%20Story%20(1995)|0|0|0|1|1|1|0|0|0|0|0|0|0|0|0|0|0|0|0
2|GoldenEye (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?GoldenEye%20(1995)|0|1|1|0|0|0|0|0|0|0|0|0|0|0|0|0|1|0|0
3|Four Rooms (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Four%20Rooms%20(1995)|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|1|0|0
4|Get Shorty (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Get%20Shorty%20(1995)|0|1|0|0|0|1|0|0|1|0|0|0|0|0|0|0|0|0|0
5|Copycat (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Copycat%20(1995)|0|0|0|0|0|0|1|0|1|0|0|0|0|0|0|0|1|0|0


------------

In [108]:
superhero_schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("name", StringType(), True),
])

In [109]:
!{"head -n 5 Marvel+Names"}

In [110]:
!{"head -n 5 Marvel+Graph"}

In [111]:
names = spark.read.schema(superhero_schema).option("sep", " ").csv("Marvel+Names")
names.show(5)

+---+--------------------+
| id|                name|
+---+--------------------+
|  1|24-HOUR MAN/EMMANUEL|
|  2|3-D MAN/CHARLES CHAN|
|  3|    4-D MAN/MERCURIO|
|  4|             8-BALL/|
|  5|                   A|
+---+--------------------+
only showing top 5 rows



In [112]:
lines = spark.read.option("sep"," ").text("Marvel+Graph")
lines.show(5)

+--------------------+
|               value|
+--------------------+
|5988 748 1722 375...|
|5989 4080 4264 44...|
|5982 217 595 1194...|
|5983 1165 3836 43...|
|5980 2731 3712 15...|
+--------------------+
only showing top 5 rows



In [151]:
lines_id = lines.withColumn("id", func.split(func.col("value"), " ")[0]).sort("id")
lines_id.show(5)

+--------------------+----+
|               value|  id|
+--------------------+----+
|1 1999 6471 6463 ...|   1|
|10 3716 1496 3712...|  10|
|100 5432 3554 311...| 100|
|1000 1165 4898 15...|1000|
|1001 199 2293 503...|1001|
+--------------------+----+
only showing top 5 rows



In [149]:
lines_conn = lines_id.withColumn("conn", func.size(func.split(func.col("value"), " "))-1)
lines_conn.show(5)

print(lines_conn.count())

+--------------------+----+----+
|               value|  id|conn|
+--------------------+----+----+
|1 1999 6471 6463 ...|   1|   6|
|10 3716 1496 3712...|  10| 188|
|100 5432 3554 311...| 100|   8|
|1000 1165 4898 15...|1000|  31|
|1001 199 2293 503...|1001|  41|
+--------------------+----+----+
only showing top 5 rows

6589


In [217]:
lines_group = lines_conn.groupBy("id").agg(func.sum("conn").alias("conn")).sort("id")
lines_group.show(5)
print(lines_group.count())

+----+----+
|  id|conn|
+----+----+
|   1|   6|
|  10| 188|
| 100|   8|
|1000|  31|
|1001|  41|
+----+----+
only showing top 5 rows

6486


In [218]:
most_popular = lines_group.sort(func.col("conn").desc())
most_popular.show(5)

+----+----+
|  id|conn|
+----+----+
| 859|1937|
|5306|1745|
|2664|1532|
|5716|1429|
|6306|1397|
+----+----+
only showing top 5 rows



In [219]:
most_popular.first().id

'859'

In [220]:
most_popular_name = names.filter(func.col("id") == most_popular.first().id).select("name")
most_popular_name.first()

Row(name='CAPTAIN AMERICA')

In [229]:
min_conn = lines_group.agg(func.min("conn")).first()[0]
most_obscure = lines_group.filter(func.col("conn") == min_conn)
most_obscure.show()

+----+----+
|  id|conn|
+----+----+
|1089|   1|
|1408|   1|
|1841|   1|
|2117|   1|
|2139|   1|
|2911|   1|
|3014|   1|
|3298|   1|
|3489|   1|
|3490|   1|
|4517|   1|
|4602|   1|
| 467|   1|
|4784|   1|
|4945|   1|
|5028|   1|
| 577|   1|
|6411|   1|
| 835|   1|
+----+----+



In [231]:
name_obscure = most_obscure.join(names, "id")
name_obscure.show(50)

+----+----+--------------------+
|  id|conn|                name|
+----+----+--------------------+
| 467|   1|        BERSERKER II|
| 577|   1|              BLARE/|
|3490|   1|MARVEL BOY II/MARTIN|
|3489|   1|MARVEL BOY/MARTIN BU|
|2139|   1|      GIURESCU, RADU|
|1089|   1|       CLUMSY FOULUP|
|1841|   1|              FENRIS|
|4517|   1|              RANDAK|
|5028|   1|           SHARKSKIN|
| 835|   1|     CALLAHAN, DANNY|
|1408|   1|         DEATHCHARGE|
|4784|   1|                RUNE|
|4945|   1|         SEA LEOPARD|
|4602|   1|         RED WOLF II|
|6411|   1|              ZANTOR|
|3014|   1|JOHNSON, LYNDON BAIN|
|3298|   1|          LUNATIK II|
|2911|   1|                KULL|
|2117|   1|GERVASE, LADY ALYSSA|
+----+----+--------------------+

